# 简单的语义内核代理教程

只需几个步骤，即可学习使用语义内核构建 AI 代理。本教程涵盖了基本要素：创建代理、添加工具和管理对话。

语义内核代理中有三个基本组件：
1. 代理类：所有代理类型都继承自此类。代理类型包括 ChatCompletionAgent（使用标准聊天完成 API）、OpenAIAssistantAgent（利用 OpenAI Assistant API 和内置工具）、AzureAIAgent（与 Azure AI 服务集成以用于企业场景）、CopilotStudioAgent（连接到 Microsoft Copilot Studio 工作流）。
2. 代理线程：这处理对话历史和状态的维护方式。这很重要，因为代理需要来自先前消息的上下文来做出明智的决定。有两种方法：
    1. 服务管理状态：像 Azure AI 这样的代理服务将对话历史存储在服务器端，并通过线程 ID 访问。
    2. 应用程序管理状态：您的应用程序维护完整的聊天历史，并在每次调用时将其传递给代理。
3. 代理编排：该框架提供了预构建的模式，用于协调多个代理以处理单个代理无法有效管理的复杂工作流。
    1. 顺序：代理按顺序一个接一个地执行。这就像文档处理管道。
    2. 并发：多个代理同时工作。就像客户查询处理（计费代理和客户代理并行工作）。
    3. 移交：代理根据专业化相互传递控制权。就像客户服务分流给专家代理。
    4. 群聊：代理在共享对话中协作。就像与领域专家进行项目规划。

代理利用语义内核的插件系统来访问工具、数据库等。

代理也可以使用 YAML 定义。

## 设置

安装所需的包并配置您的环境：

In [ ]:
# Install packages
!pip install semantic-kernel python-dotenv

# Import everything we need
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

print("✅ Setup complete!")

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Check if API key is configured
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("⚠️  Please add OPENAI_API_KEY to your .env file!")
    print("   Create a .env file with: OPENAI_API_KEY=your_actual_key_here")
else:
    print("✅ API key configured")

## 步骤 1：创建一个简单的代理

让我们从基础开始——一个可以聊天的代理。我们只是让内核（我们的协调器）访问聊天服务，该服务利用了 OpenAI 的聊天完成 API 端点。我们稍后会添加更多工具。

In [ ]:
# Create service and kernel
chat_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    api_key=OPENAI_API_KEY
)

kernel = Kernel()
kernel.add_service(chat_service)

# Create a basic agent
agent = ChatCompletionAgent(
    kernel=kernel,
    name="Assistant",
    instructions="You are a helpful and friendly assistant."
)

# Test it
response = await agent.get_response("Hello! What can you help me with?")
print(f"🤖 {agent.name}: {response.content}")

## 步骤 2：添加工具（函数）

现在让我们给我们的代理一些有用的功能。我们将给它一个天气函数和一个计算器。这两个都是我们在代码中定义的函数，并将它们作为工具提供给内核。

In [ ]:
# Define useful functions
@kernel_function(description="Get current weather for a city")
def get_weather(city: str) -> str:
    """Mock weather function - replace with real API call."""
    weather_data = {
        "london": "Cloudy, 15°C",
        "paris": "Sunny, 22°C", 
        "tokyo": "Rainy, 18°C",
        "new york": "Partly cloudy, 20°C"
    }
    return weather_data.get(city.lower(), f"Weather data not available for {city}")

@kernel_function(description="Calculate simple math expressions")
def calculate(expression: str) -> str:
    """Safe calculator for basic math."""
    try:
        # Only allow basic math operations for safety
        allowed_chars = "0123456789+-*/(). "
        if all(c in allowed_chars for c in expression):
            result = eval(expression)
            return f"{expression} = {result}"
        else:
            return "Sorry, I can only do basic math operations (+, -, *, /, parentheses)"
    except:
        return "Sorry, I couldn't calculate that. Please check your expression."

# Add functions to kernel
kernel.add_function(plugin_name="tools", function=get_weather)
kernel.add_function(plugin_name="tools", function=calculate)

# Create enhanced agent
enhanced_agent = ChatCompletionAgent(
    kernel=kernel,
    name="SmartAssistant",
    instructions="""
    You are a helpful assistant with weather and calculator capabilities.
    
    - Use get_weather when asked about weather in specific cities
    - Use calculate for math problems
    - Be friendly and explain what you're doing
    """
)

print("✅ Enhanced agent created with tools!")

## 步骤 3：使用工具测试代理

让我们看看我们的代理如何使用它的工具：

In [ ]:
# Test weather function
print("🌤️ Testing weather function:")
response = await enhanced_agent.get_response("What's the weather in London?")
print(f"🤖 {enhanced_agent.name}: {response.content}\n")

# Test calculator function  
print("🧮 Testing calculator function:")
response = await enhanced_agent.get_response("What's 25 * 4 + 10?")
print(f"🤖 {enhanced_agent.name}: {response.content}\n")

# Test general conversation
print("💬 Testing general conversation:")
response = await enhanced_agent.get_response("Tell me a fun fact about AI")
print(f"🤖 {enhanced_agent.name}: {response.content}")

## 步骤 4：带记忆的对话

对于能够记住先前消息的对话。我们将使用语义内核开箱即用的内存管理，但这将取决于上下文窗口。

内存问题可以通过总结过去的对话或使用像 RAG 这样的长期记忆来解决。还有一种学习记忆，我们希望代理从所有过去的交互中学习，这也是通过 RAG 实现的，我们存储成功的解决方案示例，并在类似情况下检索它们。

In [ ]:
async def chat_with_memory():
    """Demonstrate conversation with memory."""
    
    print("💭 Conversation with Memory Demo")
    print("=" * 40)
    
    # Messages that build on each other
    messages = [
        "Hi! I'm planning a trip to Paris.",
        "What's the weather like there?",
        "That sounds nice! Can you calculate 150 * 7 for my budget?",
        "Perfect, that should cover my week there. Thanks!"
    ]
    
    thread = None  # This will store conversation history
    
    for msg in messages:
        print(f"👤 User: {msg}")
        
        # Agent remembers previous messages through the thread
        response = await enhanced_agent.get_response(messages=msg, thread=thread)
        print(f"🤖 {enhanced_agent.name}: {response.content}\n")
        
        # Update thread to keep conversation history
        thread = response.thread

# Run the conversation demo
await chat_with_memory()

## 步骤 5：查看工具的实际操作（高级）

确切地观察当您的代理使用工具时会发生什么：

In [ ]:
async def show_tool_usage():
    """Show detailed tool execution."""
    
    print("🔧 Tool Usage Demo")
    print("=" * 25)
    
    # Callback to see tool calls
    async def log_tool_calls(message):
        from semantic_kernel.contents import FunctionCallContent, FunctionResultContent
        
        for item in message.items or []:
            if isinstance(item, FunctionCallContent):
                print(f"  🔧 Calling: {item.name}({item.arguments})")
            elif isinstance(item, FunctionResultContent):
                print(f"  ✅ Result: {item.result}")
    
    user_input = "What's the weather in Tokyo and what's 15 + 27?"
    print(f"👤 User: {user_input}\n")
    
    # Use invoke to see intermediate steps
    async for response in enhanced_agent.invoke(
        messages=user_input,
        on_intermediate_message=log_tool_calls
    ):
        print(f"\n🤖 Final Response: {response.content}")

# Run the tool usage demo
await show_tool_usage()

## 步骤 6：流式响应

对于实时响应（如 ChatGPT）：

In [ ]:
async def streaming_demo():
    """Show streaming responses."""
    
    print("\n🌊 Streaming Demo")
    print("=" * 20)
    
    print("👤 User: Write a short poem about coding")
    print("🤖 Assistant: ", end="", flush=True)
    
    # Stream response word by word
    async for chunk in enhanced_agent.invoke_stream(
        messages="Write a short poem about coding"
    ):
        print(chunk.content, end="", flush=True)
    
    print("\n")  # New line when done

# Run streaming demo
await streaming_demo()


## 总结：您学到了什么

In [ ]:
print("🎓 What You've Built:")
print("=" * 30)

summary = [
    "✅ Basic AI agent with OpenAI",
    "✅ Custom tools/functions for weather and math", 
    "✅ Conversation memory management",
    "✅ Tool execution monitoring",
    "✅ Real-time streaming responses"
]

for item in summary:
    print(item)

print("\n🚀 Key Concepts:")
concepts = {
    "Agent": "AI that can reason, remember, and use tools",
    "Kernel": "Manages AI services and functions",
    "Functions": "Tools that extend agent capabilities", 
    "Thread": "Maintains conversation history",
    "Streaming": "Real-time response generation"
}

for concept, description in concepts.items():
    print(f"• {concept}: {description}")

print("\n💡 Next Steps:")
print("• Try different models (gpt-4, gpt-3.5-turbo)")
print("• Create custom functions for your use case")
print("• Explore multi-agent conversations")
print("• Add guardrails for production safety")

## 步骤 7：顺序代理编排

现在让我们看看多个代理如何在一个管道中协同工作——每个代理处理前一个代理的输出。请注意语义内核为我们实现这一点是多么容易：

In [ ]:
# Import orchestration components
from semantic_kernel.agents import SequentialOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.contents import ChatMessageContent

print("🔗 Setting up Sequential Agent Pipeline")
print("=" * 45)

In [ ]:
# Create specialized agents for a marketing pipeline
def create_marketing_pipeline():
    """Create three agents that work together sequentially."""
    
    # Agent 1: Extract key information
    concept_extractor = ChatCompletionAgent(
        name="ConceptExtractor",
        instructions="""
        You are a marketing analyst. Given a product description, identify:
        - Key features (bullet points)
        - Target audience 
        - Unique selling points
        
        Format your output clearly with headers.
        """,
        kernel=kernel
    )
    
    # Agent 2: Write marketing copy
    copywriter = ChatCompletionAgent(
        name="Copywriter", 
        instructions="""
        You are a marketing copywriter. Take the analysis provided and write 
        compelling marketing copy (around 100-150 words). Make it engaging 
        and highlight the key benefits. Output just the marketing copy.
        """,
        kernel=kernel
    )
    
    # Agent 3: Polish and format
    editor = ChatCompletionAgent(
        name="Editor",
        instructions="""
        You are an editor. Take the marketing copy and polish it:
        - Fix grammar and clarity
        - Ensure consistent tone
        - Make it more compelling
        - Output the final polished version
        """,
        kernel=kernel
    )
    
    return [concept_extractor, copywriter, editor]

# Create the pipeline
marketing_agents = create_marketing_pipeline()
print(f"✅ Created {len(marketing_agents)} specialized agents:")
for agent in marketing_agents:
    print(f"   • {agent.name}")

In [ ]:
# Set up callback to see each agent's work
def agent_callback(message: ChatMessageContent) -> None:
    """Show what each agent produces."""
    print(f"\n🤖 {message.name}:")
    print("-" * 30)
    print(message.content)
    print()

# Create the sequential orchestration
sequential_pipeline = SequentialOrchestration(
    members=marketing_agents,
    agent_response_callback=agent_callback
)

print("🔗 Sequential pipeline created!")

In [ ]:
# Run the sequential pipeline
async def run_marketing_pipeline():
    """Execute the sequential agent pipeline."""
    
    print("🚀 Running Marketing Pipeline")
    print("=" * 35)
    
    # Start the runtime
    runtime = InProcessRuntime()
    runtime.start()
    
    try:
        # Input: Product description
        product_description = (
            "A smart water bottle with temperature display, "
            "app connectivity, hydration reminders, and "
            "leak-proof design. Made from BPA-free materials."
        )
        
        print(f"📝 Input Product: {product_description}\n")
        print("Processing through pipeline...")
        
        # Run the sequential orchestration
        result = await sequential_pipeline.invoke(
            task=product_description,
            runtime=runtime
        )
        
        # Get final result
        final_output = await result.get(timeout=30)
        
        print("🎯 FINAL MARKETING COPY:")
        print("=" * 40)
        print(final_output)
        
    finally:
        # Clean up
        await runtime.stop_when_idle()

# Execute the pipeline
await run_marketing_pipeline()

In [ ]:
# Quick demo with a different product
async def quick_pipeline_demo():
    """Quick demo with another product."""
    
    print("\n🔄 Quick Pipeline Demo #2")
    print("=" * 30)
    
    runtime = InProcessRuntime()
    runtime.start()
    
    try:
        # Different product
        product = "Wireless earbuds with 30-hour battery, noise cancellation, and workout-proof design"
        
        print(f"📝 Product: {product}")
        
        # Simple pipeline without detailed logging
        simple_pipeline = SequentialOrchestration(members=marketing_agents)
        
        result = await simple_pipeline.invoke(
            task=product,
            runtime=runtime
        )
        
        final_copy = await result.get(timeout=30)
        print(f"\n🎯 Final Copy:\n{final_copy}")
        
    finally:
        await runtime.stop_when_idle()

# Run quick demo
await quick_pipeline_demo()


## 总结：您学到了什么

In [ ]:
print("🎓 Complete Tutorial Summary:")
print("=" * 35)

summary = [
    "✅ Basic AI agent with OpenAI",
    "✅ Custom tools/functions for weather and math", 
    "✅ Conversation memory management",
    "✅ Tool execution monitoring",
    "✅ Real-time streaming responses",
    "✅ Sequential agent orchestration"
]

for item in summary:
    print(item)

print("\n🚀 Key Concepts:")
concepts = {
    "Agent": "AI that can reason, remember, and use tools",
    "Kernel": "Manages AI services and functions",
    "Functions": "Tools that extend agent capabilities", 
    "Thread": "Maintains conversation history",
    "Streaming": "Real-time response generation",
    "Sequential Orchestration": "Pipeline where agents process output sequentially"
}

for concept, description in concepts.items():
    print(f"• {concept}: {description}")

print("\n💡 Agent Patterns:")
patterns = [
    "Single Agent: One agent handles entire workflow",
    "Sequential: Agents work in pipeline (A → B → C)",
    "Concurrent: Multiple agents work simultaneously", 
    "Manager: Central agent coordinates specialists",
    "Handoff: Agents pass control to each other"
]

for pattern in patterns:
    print(f"• {pattern}")

print("\n🎯 When to Use Sequential Orchestration:")
use_cases = [
    "• Document processing (extract → summarize → format)",
    "• Content creation (research → write → edit)", 
    "• Data analysis (collect → analyze → visualize)",
    "• Code review (analyze → suggest → validate)"
]

for use_case in use_cases:
    print(use_case)

---

**🔑 关键要点：**
- **单一代理**：非常适合简单的任务和学习
- **顺序编排**：非常适合多步骤工作流，其中每个步骤都建立在前一个步骤的基础上
- **专业化**：每个代理都专注于自己最擅长的事情
- **管道优势**：通过专业化处理获得更好的质量
- **实际应用**：文档处理、内容创建、数据分析

本教程涵盖了从基本代理到复杂的多代理管道的所有内容！